# Enrichissement des données géographiques

In [1]:
import sys
import numpy as np
import pandas as pd
import openrouteservice as ors

## Import du dataset

In [2]:
data = pd.read_pickle('store/after_analyze.pkl.xz')
data.reset_index(inplace=True, drop=True)
data.head()

,date_mutation,valeur_fonciere,adresse_numero,adresse_suffixe,adresse_nom_voie,adresse_code_voie,code_postal,nom_commune,lot1_surface_carrez,type_local,surface_reelle_bati,nombre_pieces_principales,surface_terrain,longitude,latitude
0,2014-01-02,194400.0,23,NaN,BD EMILE ROMANET,2794,44100,Nantes,NaN,Appartement,84.0,4,NaN,-1.603261,47.209692
1,2014-01-02,107000.0,11,NaN,RUE DU DOCTEUR ALFRED CORLAY,0882,44800,Saint-Herblain,45.80,Appartement,46.0,2,NaN,-1.644254,47.207462
2,2014-01-09,208154.0,38,NaN,RUE DE LA PLANCHE AU GUE,5555,44300,Nantes,103.64,Appartement,103.0,5,NaN,-1.519753,47.272364
3,2014-01-06,79000.0,8,NaN,RUE DES CARMELITES,1436,44000,Nantes,26.45,Appartement,25.0,1,NaN,-1.551322,47.216626
4,2014-01-02,335000.0,23,NaN,RUE DES CANARIS,1390,44300,Nantes,NaN,Maison,118.0,5,562.0,-1.506824,47.231122


## Recherche des points d'interêt

In [3]:
ISOCHRONE_CLIENT = ors.Client(base_url='http://192.168.1.250:9090/ors')
POIS_CLIENT = ors.Client(base_url='http://192.168.1.250:5000')

ISOCHRONE_PROFILE = 'foot-walking'
ISOCHRONE_NB_MINS = 15

TEST_LOCATION = [[-1.603261,47.209692]]

In [4]:
def get_isochrone(client, location, profile='driving-car',nb_mins=5):
    params = {
        'profile':profile,
        'locations':location,
        'range':[nb_mins * 60],
        'range_type':'time'
    }
    return client.isochrones(**params)

In [5]:
def get_pois(client, geometry, request='pois'):
    params = {
        'request':request,
        'geojson':geometry
    }
    return client.places(**params)

In [6]:
def add_pois(data, profile, nb_mins, isochrone_client=ISOCHRONE_CLIENT, pois_client=POIS_CLIENT):
    results = pd.DataFrame(dtype=np.int8)
    print(data.shape)
    for i, row in data.iterrows():
        location = [[row.longitude, row.latitude]]
        isochrome = get_isochrone(isochrone_client, location, profile, nb_mins)
        geometry = isochrome['features'][0]['geometry']
        stat_pois = get_pois(pois_client, geometry, 'stats')['places']
        stat_pois.pop('total_count', None)
        tmp = dict()
        for cat, children in stat_pois.items():
            for scat, info in children['categories'].items():
                tmp['{}/{}'.format(cat,scat)] = info['count']
        pois_place = pd.concat([row, pd.Series(tmp)])
        pois_place = pois_place.to_frame().T
        results = pd.concat([results, pois_place], sort=False, ignore_index=True)
        
        sys.stdout.write('\r')
        sys.stdout.write('{}%' .format(round(100*i/data.shape[0],2)))
        sys.stdout.flush()
        
    return results

In [7]:
iso = get_isochrone(ISOCHRONE_CLIENT, TEST_LOCATION, ISOCHRONE_PROFILE, ISOCHRONE_NB_MINS)
get_pois(POIS_CLIENT, iso['features'][0]['geometry'], 'stats')

{'places': {'total_count': 2938,
  'arts_and_culture': {'group_id': 130,
   'categories': {'library': {'count': 18, 'category_id': 133},
    'place_of_worship': {'count': 10, 'category_id': 135}},
   'total_count': 28},
  'education': {'group_id': 150,
   'categories': {'kindergarten': {'count': 54, 'category_id': 153},
    'school': {'count': 75, 'category_id': 156}},
   'total_count': 129},
  'facilities': {'group_id': 160,
   'categories': {'bench': {'count': 188, 'category_id': 162},
    'clock': {'count': 64, 'category_id': 164},
    'defibrillator': {'count': 20, 'category_id': 165},
    'drinking_water': {'count': 3, 'category_id': 166},
    'fire_hydrant': {'count': 16, 'category_id': 167},
    'post_box': {'count': 32, 'category_id': 171},
    'recycling': {'count': 123, 'category_id': 172},
    'shelter': {'count': 167, 'category_id': 175},
    'toilets': {'count': 4, 'category_id': 179},
    'waste_basket': {'count': 120, 'category_id': 180},
    'waste_disposal': {'count': 

In [8]:
df = add_pois(data, ISOCHRONE_PROFILE, ISOCHRONE_NB_MINS)
df.to_pickle('store/with_poi.pkl.xz')

(38872, 15)
0.08%

KeyboardInterrupt: 